# Policy Gradient on CartPole-v0
Based on: http://kvfrans.com/simple-algoritms-for-solving-cartpole/

In [36]:
import matplotlib.pyplot as plt
%matplotlib inline 

import gym
import numpy as np
import time
import tensorflow as tf

# Parameters
timeSteps = 200 # Time steps per episode
observationSpace = 4 # Size of CartPole-v0 observation vector
actionSpace = 2 # Size of CartPole-v0 action vector
pglearningRate = 0.005 # policy gradient optimizer learning rate 
vglearningRate = 0.01 # value function optimizer learning rate 
numEpisodes = 3000 # Number of training episodes for value function - found experimentally
discountFactor = 0.97 # For calculating discounted MC return 
batchSize = 420 # Batch size for gradient ascent

In [11]:
# Setting up the CartPole-v0 environment
env = gym.make('CartPole-v0')
env.reset()
for _ in range(1000):
    env.render()
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done == True:
        print('Done on timestep:', _)
        print('Example of action:', action)
        print('Example of observation:', observation)
        break 
env.render(close=True)

[2017-12-15 21:43:19,508] Making new env: CartPole-v0


Done on timestep: 11
Example of action: 0
Example of observation: [-0.14690084 -0.78639372  0.23155486  1.4895879 ]


**run_pg_parameters(env, parameters)** takes policy gradient weights and environment as input and returns total reward for nonlinear approximation. 
**run_10_pg_trials(env, parameters)** takes policy gradient weights and environment as input and prints the results of running the environment 100 times for a 3 trials with the weights.   
**softmax(x)** is taken from here: https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python

In [55]:
def run_pg_parameters(env, parameters):
    observation = env.reset()
    totalReward = 0
    for i in range(timeSteps):
        # Choose action stochastically based on pg parameters
        probabilityActionVector = softmax(np.matmul(observation, parameters))
        randomNumber = np.random.uniform(0,1)
        if randomNumber < probabilityActionVector[0]:
            action = 0
        else: 
            action = 1
        # Take a step in the environment
        observation, reward, done, info = env.step(action)
        # Append non-discounted cumulative reward 
        totalReward += reward 
        # If done, end the episode 
        if done:
            break 
    return totalReward 

def run_10_pg_trials(env, parameters): 
    print('Using the following parameters: ', '\n', parameters)
    for i in range(10):
        overallReward = []
        for j in range(100):
            episodeReward = run_pg_parameters(env, parameters)
            overallReward.append(episodeReward)
        print('Trial', i, ': The average reward over 100 consecutive trials is:', np.mean(overallReward), 'and the maximum reward is:', np.max(overallReward))

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

For CartPole-v0, the policy is approximated as a neural network containing a set of weights shape [4,2] - 2 columns of weights corresponding to 2 actions, and 4 rows corresponding to the shape of the observations in the env. The policy recieves inputs of states and actions, and outputs the probability of choosing either action. Gradient ASCENT is used to update the weights to reflect better probabilities for choosing actions.   

The value function is also approximated as a neural network containing a set of weights in a hidden layer for computing the value of a state (not a state-action pair, just the state). The advantage function is the difference between future return Monte-Carlo style and the computed value from the current network. Gradient DESCENT is then used to update the weights to reflect closer to actual values of the states.  

Something to remember:   
**Gradient Ascent** = optimizing negative of the loss (which is really positive)   
**Gradient Descent** = optimizing positive of the loss (which is really negative)

In [37]:
# Policy Gradient Approximation
policy_gradient = tf.Graph()
with policy_gradient.as_default(): 
    # Inputs
    pgstate = tf.placeholder(tf.float32, [None, observationSpace])
    pgaction = tf.placeholder(tf.float32, [None, actionSpace])
    pgadvantages = tf.placeholder(tf.float32,[None,1])
    # Parameters 
    policyWeights = tf.get_variable("policyWeights", [observationSpace, actionSpace])
    # Output Operations 
    linear = tf.matmul(pgstate, policyWeights)
    probabilities = tf.nn.softmax(linear) 
    # Output
    processedProbabilities = tf.reduce_sum(tf.multiply(probabilities, pgaction), reduction_indices = [1])
    # Gradient Ascent Operations
    logProbabilities = tf.log(processedProbabilities)
    eligibility = logProbabilities * pgadvantages 
    pgloss = (-1) * tf.reduce_sum(eligibility)
    pgoptimizer = tf.train.RMSPropOptimizer(pglearningRate).minimize(pgloss)
    
# Value Function Approximation    
value_function = tf.Graph()
with value_function.as_default():
    # Inputs 
    vlstate = tf.placeholder(tf.float32, [None, observationSpace])
    # Input Operation
    weight1 = tf.Variable(tf.zeros([4,10]))
    bias1 = tf.Variable(tf.zeros([10]))
    # Hidden Layer 
    hidden1 = tf.nn.relu(tf.matmul(vlstate, weight1) + bias1)
    # Hidden Layer Operations
    weight2 = tf.Variable(tf.zeros([10,1]))
    bias2 = tf.Variable(tf.zeros([1]))
    # Output Operations
    calculated = tf.matmul(hidden1, weight2) + bias2
    # Another input representing discounted MC return
    newValues = tf.placeholder(tf.float32, [None,1])
    # Output 
    difference = calculated - newValues
    # Gradient Descent Operations
    vlloss = tf.nn.l2_loss(difference)
    vloptimizer = tf.train.RMSPropOptimizer(vglearningRate).minimize(vlloss)

First, we collect data from the environment using a random policy to run the episode to the terminal state. This is known as **Monte Carlo RL** because we randomly sample without bias from complete episodes in order to learn a model of the environment in the value function. These transitions will then be used to train the value function approximation.   
The improvement of the value of the starting state is demonstrated below as verification of the value function training. Since the episodes end so soon due to a bad random policy, we have to increase the number of random policy rollouts. For my set-up, **3000 rollouts** allowed the value function to converge. I set this number emperically by seeing if the starting state asymptotically approached a set value, which was about **10.5** in this case. 

In [39]:
start_time = time.time() 

masterStatesList = []
masterActionsList = []
masterAdvantagesList = [] 

with tf.Session(graph = value_function) as sess1: 
    tf.global_variables_initializer().run()
    # Sample state for value function learning verification
    testObservation = env.reset()
    # Run episodes of Cartpole to terminal state Monte-Carlo style   
    for _ in range(numEpisodes):
        observation = env.reset()
        states = []
        actions = []
        transitions = []
        advantages = [] 
        # Sample up to 200 random steps per episode or until terminal episode  
        for t in range(timeSteps):
            action = env.action_space.sample()
            # Append state 
            masterStatesList.append(observation)
            states.append(observation)
            # Append action
            actionVector = np.zeros(2)
            actionVector[action] = 1
            masterActionsList.append(actionVector)
            actions.append(actionVector)
            # Take a step in the environment
            oldObservation = observation
            observation, reward, done, info = env.step(action)
            # Record transitions
            transitions.append([oldObservation, action, reward])
            # Break if done with episode
            if done == True:
                break       
        # Train the value function
        updatedValues = []
        for index, transitionTuples in enumerate(transitions):
            observation, action, reward = transitionTuples
            # Calculate discounted MC return starting from each state and moving towards terminal state
            totalDiscountedReward = 0 
            rolloutLength = len(transitions) - index 
            for i in range(rolloutLength):
                # Calculate the discount factor
                discount = np.power(discountFactor, i)
                # transitions[i + index][2] is the third entry - [oldObservation, action, reward]
                totalDiscountedReward += (transitions[i + index][2]) * discount 
            # Append the updated values to the list
            updatedValues.append(totalDiscountedReward)
            # Append the advantage to the list
            currentValue = sess1.run(calculated, feed_dict = {vlstate: [observation]})
            advantages.append(totalDiscountedReward - currentValue)
            masterAdvantagesList.append(totalDiscountedReward - currentValue)
        # Use the list to train the value function by calling sess2.run()
        updatedValues = np.expand_dims(updatedValues, axis = 1)
        sess1.run(vloptimizer, feed_dict = {vlstate: states, newValues: updatedValues})
        # Training Log
        if _ % 1000 == 0 or _ == numEpisodes - 1:
            currentStateValue = sess1.run(calculated, feed_dict = {vlstate: [testObservation]})
            currentLoss = sess1.run(vlloss, feed_dict = {vlstate: states, newValues: updatedValues})
            print('Episode',_,'Starting state value:', round(float(currentStateValue), 2), 'Loss:', round(float(currentLoss), 2), 'Advantage:', round(float(advantages[0]), 2))        
end_time = time.time()
print('Value function training time in seconds:', end_time - start_time)

Episode 0 Starting state value: 0.03 Loss: 2558.52 Advantage: 19.97
Episode 1000 Starting state value: 6.53 Loss: 497.73 Advantage: 11.24
Episode 2000 Starting state value: 9.68 Loss: 212.39 Advantage: 6.07
Episode 2999 Starting state value: 10.21 Loss: 1305.07 Advantage: 13.58
Value function training time in seconds: 19.540735960006714


Second, we then train the policy gradient by training on batches from the state, action, and advantage data we collected earlier.

In [52]:
# Helpful tips: http://rll.berkeley.edu/deeprlcourse/f17docs/lecture_4_policy_gradient.pdf
start_time = time.time()
untrainedWeights = np.zeros((4,2))
trainedWeights = np.zeros((4,2))
with tf.Session(graph = policy_gradient) as sess2: 
    tf.global_variables_initializer().run()
    untrainedWeights = sess2.run(policyWeights)
    # Train policy gradient
    stateBatch = []
    actionBatch = []
    advantageBatch = []
    # Sample state for tracking the loss 
    testState = [masterStatesList[1]]
    testAction = np.asarray(masterActionsList[1]).reshape(1, actionSpace)
    testAdvantage = masterAdvantagesList[1]
    # Start training
    for _ in range(len(masterStatesList)):
        # Batch training
        if len(stateBatch) == batchSize: 
            stateBatchArray = np.asarray(stateBatch).reshape(batchSize, observationSpace)
            actionBatchArray = np.asarray(actionBatch).reshape(batchSize, actionSpace)
            advantageBatchArray = np.asarray(advantageBatch).reshape(batchSize, 1)
            # print(sess1.run(linear, feed_dict = {pgstate: stateBatchArray, pgaction: actionBatchArray, pgadvantages: advantageBatchArray}))
            sess2.run(pgoptimizer, feed_dict = {pgstate: stateBatchArray, pgaction: actionBatchArray, pgadvantages: advantageBatchArray})
            # Empty batch
            stateBatch = []
            actionBatch = []
            advantageBatch = []    
        # Append to batch 
        else: 
            stateBatch.append([masterStatesList[i]])
            actionBatch.append(np.asarray(masterActionsList[i]).reshape(1, actionSpace))
            advantageBatch.append(masterAdvantagesList[i]) 
        # Training log
        if _ % 10000 == 0 or _ == len(masterStatesList) - 1:
            currentProbability = sess2.run(processedProbabilities, feed_dict = {pgstate: testState, pgaction: testAction, pgadvantages: testAdvantage})
            currentLoss = sess2.run(pgloss, feed_dict = {pgstate: testState, pgaction: testAction, pgadvantages: testAdvantage})
            print('Episode',_,'Starting state chosen action:', testAction,'probability:', round(float(currentProbability), 2), 'Loss:', round(float(currentLoss), 2))
    trainedWeights = sess2.run(policyWeights)
end_time = time.time()
print('Policy gradient training time in seconds:', end_time - start_time)
print('Untrained policy parameters:', '\n', untrainedWeights)
print('Trained policy parameters:','\n' , trainedWeights)

Episode 0 Starting state chosen action: [[ 0.  1.]] probability: 0.54 Loss: 12.15
Episode 10000 Starting state chosen action: [[ 0.  1.]] probability: 0.57 Loss: 11.03
Episode 20000 Starting state chosen action: [[ 0.  1.]] probability: 0.59 Loss: 10.22
Episode 30000 Starting state chosen action: [[ 0.  1.]] probability: 0.62 Loss: 9.47
Episode 40000 Starting state chosen action: [[ 0.  1.]] probability: 0.64 Loss: 8.76
Episode 50000 Starting state chosen action: [[ 0.  1.]] probability: 0.66 Loss: 8.12
Episode 60000 Starting state chosen action: [[ 0.  1.]] probability: 0.68 Loss: 7.49
Episode 67172 Starting state chosen action: [[ 0.  1.]] probability: 0.7 Loss: 7.07
Policy gradient training time in seconds: 0.9735894203186035
Untrained policy parameters: 
 [[-0.25356054  0.23324466]
 [-0.38843489  0.17011738]
 [-0.68659639 -0.71196938]
 [-0.33538461  0.45692039]]
Trained policy parameters: 
 [[ 0.58406454 -0.60438049]
 [ 0.44919032 -0.66750789]
 [ 0.15102857 -1.54959464]
 [-1.173010

In [56]:
# Untrained weights
run_10_pg_trials(env, untrainedWeights)

Using the following parameters:  
 [[-0.25356054  0.23324466]
 [-0.38843489  0.17011738]
 [-0.68659639 -0.71196938]
 [-0.33538461  0.45692039]]
Trial 0 : The average reward over 100 consecutive trials is: 25.0 and the maximum reward is: 93.0
Trial 1 : The average reward over 100 consecutive trials is: 31.03 and the maximum reward is: 105.0
Trial 2 : The average reward over 100 consecutive trials is: 31.18 and the maximum reward is: 92.0
Trial 3 : The average reward over 100 consecutive trials is: 29.77 and the maximum reward is: 136.0
Trial 4 : The average reward over 100 consecutive trials is: 29.58 and the maximum reward is: 79.0
Trial 5 : The average reward over 100 consecutive trials is: 27.98 and the maximum reward is: 71.0
Trial 6 : The average reward over 100 consecutive trials is: 29.57 and the maximum reward is: 88.0
Trial 7 : The average reward over 100 consecutive trials is: 27.65 and the maximum reward is: 82.0
Trial 8 : The average reward over 100 consecutive trials is: 29

In [57]:
# Trained weights
run_10_pg_trials(env, trainedWeights)

Using the following parameters:  
 [[ 0.58406454 -0.60438049]
 [ 0.44919032 -0.66750789]
 [ 0.15102857 -1.54959464]
 [-1.17301035  1.29454553]]
Trial 0 : The average reward over 100 consecutive trials is: 56.22 and the maximum reward is: 131.0
Trial 1 : The average reward over 100 consecutive trials is: 52.9 and the maximum reward is: 185.0
Trial 2 : The average reward over 100 consecutive trials is: 50.39 and the maximum reward is: 124.0
Trial 3 : The average reward over 100 consecutive trials is: 47.34 and the maximum reward is: 87.0
Trial 4 : The average reward over 100 consecutive trials is: 54.38 and the maximum reward is: 142.0
Trial 5 : The average reward over 100 consecutive trials is: 51.04 and the maximum reward is: 104.0
Trial 6 : The average reward over 100 consecutive trials is: 50.89 and the maximum reward is: 135.0
Trial 7 : The average reward over 100 consecutive trials is: 48.49 and the maximum reward is: 130.0
Trial 8 : The average reward over 100 consecutive trials i

Conclusion: Clearly the trained policy parameters perform better on average. But more hyperparameter tuning is needed for the learning rate, training batch sizes, and number of training episodes. I didn't solve the environment, but I can see why policy gradients are useful for continuous control as shown above. The tutorial also showed the random policy search actually performed the best due to the small action space. For more complicated environments like humanoid or swimmer, the policy gradient will be much more useful. 